In [6]:
from __future__ import print_function

import os
import cv2
import numpy as np
np.random.seed(1337)
import sys

from keras.layers import Input
from keras_vggface.vggface import VGGFace

from vocabulary import Vocabulary

In [7]:
 input_tensor = Input(shape=(224, 224, 3))
vgg_model = VGGFace(input_tensor=input_tensor, include_top=False, pooling='avg')

58916864/58909280 [==============================] - 8s 0us/step


In [8]:
vgg_model.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)     

In [36]:
def kfold(x, k):
    ''' Split testing and training data k ways to enable k-fold
        cross-validation. '''
    interval = int(x.shape[0] / k)

    masks = []
    for i in range(1, k + 1):
        # Create mask for testing section of data.
        test_mask = np.zeros(x.shape[0], dtype=bool)
        test_mask[(i-1)*interval:i*interval] = True

        masks.append(test_mask)


    return masks


In [37]:
import numpy as np
features = np.zeros(shape=(3 ,6,512), dtype=np.float32)

In [38]:
labels = np.asarray(['red', 'blue', 'bin'])

In [39]:
feature_one = np.ndarray(shape=(6,512), dtype=np.float32)
print(feature_one)

features[0] = feature_one
feature_one[0][0] = 999
features[1] = feature_one

feature_one[0][0] = 111
features[2] = feature_one
print(features)

[[2.3123192e-10 8.4358168e-43 1.8530510e-10 ... 0.0000000e+00
  1.8624313e-10 8.4358168e-43]
 [3.6433760e-44 0.0000000e+00 1.9229085e-10 ... 8.4358168e-43
  1.8323565e-10 8.4358168e-43]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
[[[2.3123192e-10 8.4358168e-43 1.8530510e-10 ... 0.0000000e+00
   1.8624313e-10 8.4358168e-43]
  [3.6433760e-44 0.0000000e+00 1.9229085e-10 ... 8.4358168e-43
   1.8323565e-10 8.4358168e-43]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+0

In [40]:
k = 3
kmask = kfold(features, k)

print(kmask)

[array([ True, False, False]), array([False,  True, False]), array([False, False,  True])]


In [47]:

for fold in range(k):
    print('Fold: ', fold)

    # Split data based on current fold.
    train, test = split_train_test(features, labels, kmask[fold])
    print (train)
    print(test)

Fold:  0
(array([[[9.9900000e+02, 8.4358168e-43, 1.8530510e-10, ...,
         0.0000000e+00, 1.8624313e-10, 8.4358168e-43],
        [3.6433760e-44, 0.0000000e+00, 1.9229085e-10, ...,
         8.4358168e-43, 1.8323565e-10, 8.4358168e-43],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[1.1100000e+02, 8.4358168e-43, 1.8530510e-10, ...,
         0.0000000e+00, 1.8624313e-10, 8.4358168e-43],
        [3.6433760e-44, 0.0000000e+00, 1.9229085e-10, ...,
         8.4358168e-43, 1.8323565e-10, 8.4358168e-43],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.000000

In [1]:
def split_train_test(x, y, test_mask):
    x_train = x[~test_mask, ...]
    y_train = y[~test_mask, ...]
    train = (x_train, y_train)

    x_test = x[test_mask, ...]
    y_test = y[test_mask, ...]
    test = (x_test, y_test)
    return train, test